In [80]:
import pandas as pd 
import os
import xmltodict
import numpy as np
import xml.etree.ElementTree as etree
from collections import OrderedDict
#pd.options.display.max_rows  = 50
pd.options.display.max_colwidth = -1
print("successfully imported")

successfully imported


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [81]:
tree = etree.parse("/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml")
root = tree.getroot()

def iter_docs(author, elem):
    author_attr = author.attrib
    for doc in author.iter(elem):
        doc_dict = author_attr.copy()
        doc_dict.update(doc.attrib)
        doc_dict['data'] = doc.text
        yield doc_dict
        
def iter_author(etree):
    for author in etree.iter('author'):
        for row in iter_docs(author):
            yield row
            
def flatten_dict(d):
    def items():
        for key, value in d.items():
            if isinstance(value, dict):
                for subkey, subvalue in flatten_dict(value).items():
                    yield key + "." + subkey, subvalue
            else:
                yield key, value
    return OrderedDict(items())

def flatten_dict_detailed(d):
    def items():
        for key, value in d.items():
            if isinstance(value, dict):
                for subkey, subvalue in flatten_dict_detailed(value).items():
                    yield key + "." + subkey, subvalue
            if isinstance(value, list):
                for val in value:
                    if isinstance(val, str):
                        yield key, val
                    if isinstance(val, dict):
                         for subkey, subvalue in flatten_dict_detailed(val).items():
                            yield key + "." + subkey, subvalue
                    if isinstance(val, list):
                        flatten_dict_detailed(val)
            else:
                yield key, value
    
    return OrderedDict(items())

# Convert XML to dict
#TODO: create better CCD ingestion 
with open('/Users/jamesz/dev/saraswati-data-quality/sample_CDA/C-CDA_R2-1_CCD.xml', 'rb') as f:
    xml_content = xmltodict.parse(f)

# Flatten dict
flattened_xml = flatten_dict(xml_content)
agg = []

# Print in desired format
for k,v in flattened_xml.items():
    agg.append((k,v))
    
flattened_df = pd.DataFrame(agg, columns = ['tags', 'value'])


In [82]:
"""
ALLERGIES_SECTION
IMMUNIZATIONS_SECTION
MEDICATIONS_SECTION
PROBLEM_SECTION
PROCEDURES_SECTION
ENCOUNTERS_SECTION
ENCOUNTERS_SECTION_ENTRIES_OPTIONAL
RESULTS_SECTION
VITAL_SIGNS_SECTION
"""

'\nALLERGIES_SECTION\nIMMUNIZATIONS_SECTION\nMEDICATIONS_SECTION\nPROBLEM_SECTION\nPROCEDURES_SECTION\nENCOUNTERS_SECTION\nENCOUNTERS_SECTION_ENTRIES_OPTIONAL\nRESULTS_SECTION\nVITAL_SIGNS_SECTION\n'

In [83]:
# if writing seperate parser for each section 
#Allergies section
# string = '{urn:hl7-org:v3}component/{urn:hl7-org:v3}structuredBody/{urn:hl7-org:v3}component/{urn:hl7-org:v3}section'
# elem = root.findall(string)
# for x in elem:
#     if x.find('{urn:hl7-org:v3}title').text == 'ALLERGIES AND ADVERSE REACTIONS':
#         for y in x:
#             print(y.tag)
#             if y.tag == '{urn:hl7-org:v3}text':
#                 xml_string = etree.tostring(y)
#                 text_df = pd.read_html(xml_string)[0]
#             if y.tag == '{urn:hl7-org:v3}code':
#                 code_df = pd.DataFrame([y.attrib])
#             entries = [] 
#             if y.tag == '{urn:hl7-org:v3}entry':
#                 entries.append(y)

In [84]:
#list of sections in CCD
pd.DataFrame(list(iter_docs(root, '{urn:hl7-org:v3}title')))

,data
0,Patient Chart Summary
1,ADVANCE DIRECTIVES
2,ALLERGIES AND ADVERSE REACTIONS
3,ENCOUNTERS
4,FAMILY HISTORY
5,FUNCTIONAL STATUS
6,IMMUNIZATIONS
7,MEDICAL EQUIPMENT
8,MEDICATIONS
9,INSURANCE PROVIDERS


In [108]:
#@ = attribute
#. = next node
flattened_df.head(20)

,tags,value
0,ClinicalDocument.@xmlns,urn:hl7-org:v3
1,ClinicalDocument.@xmlns:xsi,http://www.w3.org/2001/XMLSchema-instance
2,ClinicalDocument.@xmlns:voc,urn:hl7-org:v3/voc
3,ClinicalDocument.@xmlns:sdtc,urn:hl7-org:sdtc
4,ClinicalDocument.realmCode.@code,US
5,ClinicalDocument.typeId.@extension,POCD_HD000040
6,ClinicalDocument.typeId.@root,2.16.840.1.113883.1.3
7,ClinicalDocument.templateId,"[{'@root': '2.16.840.1.113883.10.20.22.1.2', '@extension': '2015-08-01'}, {'@root': '2.16.840.1.113883.10.20.22.1.2'}]"
8,ClinicalDocument.id.@extension,TT988
9,ClinicalDocument.id.@root,2.16.840.1.113883.19.5.99999.1


In [86]:
#Find indexes where value is not string
nested_indexes = []
for i, val in enumerate(flattened_df.value):
    if type(val) == list:
        nested_indexes.append(i)

In [87]:
flattened_df.loc[nested_indexes].shape

(9, 2)

In [97]:
#flattened_df.loc[nested_indexes]

In [98]:
#hard coding for now
informant = pd.DataFrame(flattened_df.loc[131].value)
participant = pd.DataFrame(flattened_df.loc[186].value)
components = pd.DataFrame(flattened_df.loc[223].value)

In [90]:
#each row is another component (allergies, advance directives etc.)
components.shape

(15, 1)

In [91]:
#advanced directives section 
advance_directives = flatten_dict(components.section[0])
advance_directives_df = pd.DataFrame(advance_directives.items(), columns = ['tag', 'value'])

In [92]:
advance_directives

OrderedDict([('templateId',
              [OrderedDict([('@root', '2.16.840.1.113883.10.20.22.2.21'),
                            ('@extension', '2015-08-01')]),
               OrderedDict([('@root', '2.16.840.1.113883.10.20.22.2.21')])]),
             ('code.@code', '42348-3'),
             ('code.@codeSystem', '2.16.840.1.113883.6.1'),
             ('code.@codeSystemName', 'LOINC'),
             ('title', 'ADVANCE DIRECTIVES'),
             ('text.table.@border', '1'),
             ('text.table.@width', '100%'),
             ('text.table.thead.tr.th',
              ['Directive',
               'Description',
               'Verification',
               'Supporting Document(s)']),
             ('text.table.tbody.tr.td',
              ['Resuscitation status',
               OrderedDict([('@ID', 'AD1'), ('#text', 'Do not resuscitate')]),
               'Dr. Patricia Primary, Feb 19, 2011',
               OrderedDict([('linkHtml',
                             OrderedDict([('@href',
    

In [93]:
flatten_dict_detailed(advance_directives)

OrderedDict([('templateId.@root', '2.16.840.1.113883.10.20.22.2.21'),
             ('templateId.@extension', '2015-08-01'),
             ('code.@code', '42348-3'),
             ('code.@codeSystem', '2.16.840.1.113883.6.1'),
             ('code.@codeSystemName', 'LOINC'),
             ('title', 'ADVANCE DIRECTIVES'),
             ('text.table.@border', '1'),
             ('text.table.@width', '100%'),
             ('text.table.thead.tr.th', 'Supporting Document(s)'),
             ('text.table.tbody.tr.td', 'Dr. Patricia Primary, Feb 19, 2011'),
             ('text.table.tbody.tr.td.@ID', 'AD1'),
             ('text.table.tbody.tr.td.#text', 'Do not resuscitate'),
             ('text.table.tbody.tr.td.linkHtml.@href',
              'AdvanceDirective.b50b7910.pdf'),
             ('text.table.tbody.tr.td.linkHtml.#text', 'Advance directive'),
             ('text.table.tbody.tr.td.linkHtml',
              OrderedDict([('@href', 'AdvanceDirective.b50b7910.pdf'),
                           ('

In [94]:
#immunizations section
Immunizations = flatten_dict(components.section[5])
Immunizations_df =pd.DataFrame(flatten_dict_detailed(Immunizations).items(), columns = ['tag', 'value'])

In [111]:
Immunizations_df.head(25)

,tag,value
0,templateId.@root,2.16.840.1.113883.10.20.22.2.2.1
1,templateId.@extension,2014-06-09
2,code.@code,11369-6
3,code.@codeSystem,2.16.840.1.113883.6.1
4,code.@codeSystemName,LOINC
5,code.@displayName,History of immunizations
6,title,IMMUNIZATIONS
7,text.content.@ID,immunSect
8,text.table.@border,1
9,text.table.@width,100%


In [96]:
hi = pd.DataFrame(flatten_dict_detailed(Immunizations).items(), columns = ['tags', 'value'])
hi[hi.tags.str.contains('id', regex=True)]

,tags,value
9,text.table.@width,100%
21,entry.substanceAdministration.id.@root,de10790f-1496-4719-8fe6-f1b87b6219f7
22,entry.substanceAdministration.id,{'@root': 'de10790f-1496-4719-8fe6-f1b87b6219f7'}
50,entry.substanceAdministration.performer.assignedEntity.id.@root,2.16.840.1.113883.19.5.9999.456
51,entry.substanceAdministration.performer.assignedEntity.id.@extension,2981824
52,entry.substanceAdministration.performer.assignedEntity.id,"{'@root': '2.16.840.1.113883.19.5.9999.456', '@extension': '2981824'}"
66,entry.substanceAdministration.performer.assignedEntity.representedOrganization.id.@root,2.16.840.1.113883.19.5.9999.1394
67,entry.substanceAdministration.performer.assignedEntity.representedOrganization.id,{'@root': '2.16.840.1.113883.19.5.9999.1394'}
107,entry.substanceAdministration.entryRelationship.observation.id.@root,db734647-fc99-424c-a864-7e3cda82e703
108,entry.substanceAdministration.entryRelationship.observation.id,"{'@root': 'db734647-fc99-424c-a864-7e3cda82e703', '@extension': '45665'}"
